# Play Store App Reviews - Doc

In [1]:
from google_play_scraper import reviews, Sort
import pandas as pd
import plotly.express as px

#### Setando as variávies.

In [2]:
APP_ID = "br.com.intermedium"
LANGUAGE = "pt-br"
COUNTRY = "br"

TOTAL_ROWS_OF_DATA = 10000

#### Rodando o request e pegando os dados.

In [3]:
results, _ = reviews(
    APP_ID,
    lang=LANGUAGE,
    country=COUNTRY,
    sort=Sort.NEWEST,
    count=TOTAL_ROWS_OF_DATA,
    filter_score_with=None,
)

In [4]:
data = []

for result in results:
    review_id = result["reviewId"]
    content = result["content"]
    score = result["score"]
    thumbs_up_count = result["thumbsUpCount"]
    review_created_version = result["reviewCreatedVersion"]
    at = result["at"]
    reply_content = result["replyContent"]
    replied_at = result["repliedAt"]
    app_version = result["appVersion"]

    data.append(
        {
            "Review ID": review_id,
            "Content": content,
            "Score": score,
            "Thumbs Up Count": thumbs_up_count,
            "Review Created Version": review_created_version,
            "At": at,
            "Reply Content": reply_content,
            "Replied At": replied_at,
            "App Version": app_version,
        }
    )

df = pd.DataFrame(data)

#### Exibindo as cinco primeiras linhas dos dados.

In [5]:
df.head()

,Review ID,Content,Score,Thumbs Up Count,Review Created Version,At,Reply Content,Replied At,App Version
0,c132afbf-87c0-4dcb-97ca-d3c6bc11845b,muito bom.,5,0,11.5.3,2023-05-28 13:15:18,"Oi, tudo bem? Obrigado pelo seu feedback, fica...",2023-05-28 13:17:56,11.5.3
1,eecb4019-0d49-40c3-9241-3f17c7b1b438,Horrível. para entrar em minha conta o app sim...,1,0,11.5.2,2023-05-28 13:06:28,"Oi! Para que possamos te ajudar, entre em cont...",2023-05-28 13:18:00,11.5.2
2,0d9ec2a2-1db5-49a1-b733-83402a76f3ef,"muito bom, recomendo.",5,0,11.5.3,2023-05-28 13:02:42,"Oi, tudo bem? Obrigado pelo seu feedback, fica...",2023-05-28 13:17:55,11.5.3
3,b83f46d7-410c-489e-9e46-746f6276a6e5,Kkkkk cep nao pode ser utilizado nao entregamo...,1,0,None,2023-05-28 12:53:35,Oi! Gostaríamos de entender melhor o que está ...,2023-05-28 13:18:01,None
4,8780333e-c649-47ce-a13f-44174b76886f,Nao estou conseguindo acessar minha conta. Ped...,3,0,11.3.6,2023-05-28 12:33:40,None,NaT,11.3.6


### Gráfico de distribuição das pontuações.

* Qual é a quantidade de avaliações para 1 estrela? Devemos nos preocupar? 🤔

In [6]:
score_counts = df['Score'].value_counts()
score_counts = score_counts.sort_index()

fig = px.bar(x=score_counts.index, y=score_counts.values, title='Distribuição das pontuações.')
fig.update_layout(xaxis_title='Pontuação', yaxis_title='Contagem')

fig.show()

### Gráfico de avaliações por versão de app.
* A maioria das nossas avaliações são de versões passadas?
* Na nossa nova versão, nós já temos bastantes avaliações?

In [7]:
value_counts = df['App Version'].value_counts()

value_percentages = value_counts / len(df) * 100

filtered_values = value_counts[value_percentages >= 5]
filtered_value_percentages = value_percentages[value_percentages >= 5]

fig = px.pie(values=filtered_values, names=filtered_values.index, title="Avaliações por versão do aplicativo. (Excluído < 5%)")
fig.show()

### Gráfico de dispersão de comentários com curtidas.
* Nesses últimos tempos, nossos comentários com mais curtidas foram bem avaliados?
* Temos um comentário com 1 estrela que foi muito curtido?

In [8]:
fig = px.scatter(df, x='Score', y='Thumbs Up Count', title='Gráfico de dispersão dos comentários e suas respectivas curtidas.',
                 labels={'Score': 'Avaliação', 'Thumbs Up Count': 'Curtidas'})

fig.update_traces(marker=dict(size=10))
fig.update_layout(xaxis=dict(tickmode='linear', dtick=1))
fig.show()